<a href="https://colab.research.google.com/github/markNZed/GPT-NeoX-Colab/blob/main/notebooks/shakespeare_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/markNZed/GPT-NeoX-Colab.git
!ls

fatal: destination path 'GPT-NeoX-Colab' already exists and is not an empty directory.
GPT-NeoX-Colab	sample_data


# Cloning GPT-NeoX Repo

In [ ]:
!git clone https://github.com/EleutherAI/gpt-neox

Cloning into 'gpt-neox'...
remote: Enumerating objects: 19481, done.
remote: Counting objects: 100% (1684/1684), done.
remote: Compressing objects: 100% (812/812), done.
remote: Total 19481 (delta 1241), reused 1196 (delta 864), pack-reused 17797 (from 1)
Receiving objects: 100% (19481/19481), 113.67 MiB | 21.86 MiB/s, done.
Resolving deltas: 100% (14098/14098), done.


# Installing Dependencies

In [ ]:
%cd /content/gpt-neox
!pip install -r ./requirements/requirements.txt
# Currently deepsped rom GTP-NeoX is not compatibel with logging in torch >= 2.4
!pip install torch==2.3

/content/gpt-neox/requirements
  Cloning https://github.com/EleutherAI/DeeperSpeed.git (to revision 02e2ebf7dee6aaab3d89094ed470a4609763c742) to /tmp/pip-install-9qls9cj2/deepspeed_3d04e9c99e4345adbd8c6e0aa9d8e6e3
  Running command git clone --filter=blob:none --quiet https://github.com/EleutherAI/DeeperSpeed.git /tmp/pip-install-9qls9cj2/deepspeed_3d04e9c99e4345adbd8c6e0aa9d8e6e3
  Running command git rev-parse -q --verify 'sha^02e2ebf7dee6aaab3d89094ed470a4609763c742'
  Running command git fetch -q https://github.com/EleutherAI/DeeperSpeed.git 02e2ebf7dee6aaab3d89094ed470a4609763c742
  Running command git checkout -q 02e2ebf7dee6aaab3d89094ed470a4609763c742
  Resolved https://github.com/EleutherAI/DeeperSpeed.git to commit 02e2ebf7dee6aaab3d89094ed470a4609763c742
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/EleutherAI/lm_dataformat.git (to revision 4eec05349977071bf67fc072290b95e31c8dd836) to /tmp/p

# Preparing Custom Dataset

In [ ]:
%cd /content/gpt-neox
!python prepare_data.py -d ./data

[Errno 2] No such file or directory: 'gpt-neox'
/content/gpt-neox
--2024-10-29 06:51:07--  https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-vocab.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 16.15.216.179, 16.182.71.176, 52.217.81.142, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|16.15.216.179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1042301 (1018K) [application/json]
Saving to: ‘./data//gpt2-vocab.json’

./data//gpt2-vocab. 100%[===================>]   1018K   867KB/s    in 1.2s    

2024-10-29 06:51:08 (867 KB/s) - ‘./data//gpt2-vocab.json’ saved [1042301/1042301]

--2024-10-29 06:51:08--  https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-merges.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 16.15.216.179, 16.182.71.176, 52.217.81.142, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|16.15.216.179|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 456318 (446K) [text/plain]
Saving to: 

# Downloading TinyShakespear Dataset

In [ ]:
%cd /content/gpt-neox/data
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-10-29 06:55:31--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2024-10-29 06:55:31 (22.4 MB/s) - ‘input.txt’ saved [1115394/1115394]



# Tokenizing Dataset

In [ ]:
import json

# Generate a list of dictionaries
lines = []
with open("input.txt", encoding="utf8") as f:
    for line in f.read().splitlines():
        if line:
            lines.append({"text": line})

# Convert to a list of JSON strings
json_lines = [json.dumps(l) for l in lines]

# Join lines and save to .jsonl file
json_data = '\n'.join(json_lines)
with open('tineyhsakespear.jsonl', 'w') as f:
    f.write(json_data)

In [ ]:
%cd /content/gpt-neox

/content/gpt-neox


In [ ]:
!python tools/datasets/preprocess_data.py \
    --input ./data/tineyhsakespear.jsonl \
    --output-prefix ./data/tinyshakespeare \
    --vocab-file ./data/gpt2-vocab.json \
    --merge-file ./data/gpt2-merges.txt \
    --dataset-impl mmap \
    --tokenizer-type GPT2BPETokenizer \
    --append-eod


[2024-10-29 06:58:13,283] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
> building GPT2BPETokenizer tokenizer ...
 > padded vocab (size: 50257) with 47 dummy tokens (new size: 50304)
Vocab size: 50257
Output prefix: ./data/tinyshakespeare
> building GPT2BPETokenizer tokenizer ...
 > padded vocab (size: 50257) with 47 dummy tokens (new size: 50304)
Processed 32700 documents (7347.61 docs/s, 0.23 MB/s).: : 32700it [00:04, 7318.35it/s]


In [ ]:
%load_ext tensorboard
!python ./deepy.py train.py -d configs /content/5M.yml local_setup.yml

Streaming output truncated to the last 5000 lines.
[2024-10-29 08:36:13,700] [INFO] [logging.py:96:log_dist] [Rank 0] comm op: all_reduce | time (ms): 0.05 | msg size: 55.15 MB | algbw (Gbps): 18254.55 | busbw (Gbps): 0.00
[2024-10-29 08:36:13,705] [INFO] [logging.py:96:log_dist] [Rank 0] comm op: all_reduce | time (ms): 0.15 | msg size: 1.0 B | algbw (Gbps): 0.00 | busbw (Gbps): 0.00
[2024-10-29 08:36:13,709] [INFO] [logging.py:96:log_dist] [Rank 0] comm op: all_reduce | time (ms): 0.14 | msg size: 4.0 B | algbw (Gbps): 0.00 | busbw (Gbps): 0.00
[2024-10-29 08:36:13,711] [INFO] [logging.py:96:log_dist] [Rank 0] comm op: all_reduce | time (ms): 0.13 | msg size: 4.0 B | algbw (Gbps): 0.00 | busbw (Gbps): 0.00
[2024-10-29 08:36:13,719] [INFO] [logging.py:96:log_dist] [Rank 0] comm op: all_gather_into_tensor | time (ms): 0.04 | msg size: 55.12 MB | algbw (Gbps): 12544.00 | busbw (Gbps): 0.00
[2024-10-29 08:36:13,720] [INFO] [logging.py:96:log_dist] [Rank 0] comm op: all_gather_into_tensor

# Inference
The output of the model will be stored in `sample_outpu.txt` file

In [ ]:
!python ./deepy.py generate.py -d configs /content/GPT-NeoX-Colab/configs/shakespeare.yml local_setup.yml text_generation.yml

[2024-10-29 08:43:51,870] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)
NeoXArgs.from_ymls() ['/content/5M.yml', 'configs/local_setup.yml', 'configs/text_generation.yml']
INFO:root:NeoXArgs.calculate_derived() Total number of GPUs determined to be: 1
-------------------- arguments --------------------
  attention_config ................ ['global', 'global', 'global', 'global']updated
  batch_size ...................... 4...........................updated
  checkpoint_activations .......... True........................updated
  checkpoint_factor ............... 1000........................updated
  config_files .................... {'5M.yml': '{\n  "pipe_parallel_size": 1,\n  "model_parallel_size": 1,\n\n  # model settings\n  "num_layers": 4,\n  "hidden_size": 256,\n  "num_attention_heads": 4,\n  "seq_length": 1024,\n  "max_position_embeddings": 1024,\n  "pos_emb": "rotary",\n  "no_weight_tying": true,\n  "gpt_j_residual": false,\n  "outpu

# Storing Repo with Checkpoints in Google Drive


In [ ]:
import shutil

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define source and destination paths
source_folder = '/content/gpt-neox'
destination_folder = '/content/drive/MyDrive/gpt-neox'

# Copy the folder
shutil.copytree(source_folder, destination_folder)

print("Folder copied successfully to Google Drive!")


Mounted at /content/drive
Folder copied successfully to Google Drive!
